# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

Для второй спецификации считаем бутстраповские доверительные интервалы

In [1]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)


Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:utils':

    timestamp



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"пакет 'plotly' был собран под R версии 4.4.2"

Присоеди

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [2]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,C1M_School_closing_i,C1M_School_closing_j,population_density_i,population_density_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,1.183562,1.49315071,248.620682,4.055137,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,1.290411,0.07671233,8.838488,89.445110,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,1.584700,1.98087430,26.542530,531.109006,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,2.153005,2.40437150,108.343854,98.596161,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,NA,1.00000000,179.857692,63.377090,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,2.000000,0.27945206,122.707272,1660.971875,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [6]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(v) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year + i^j,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  year: 3,  i^j: 20,608
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error  t value Pr(>|t|)    
log(deaths_i)                        -0.015511   0.007764 -1.99787 0.047289 *  
log(deaths_j)                        -0.007561   0.007003 -1.07965 0.281790    
C8EV_International_travel_controls_i -0.038251   0.019968 -1.91564 0.057051 .  
C8EV_International_travel_controls_j -0.027171   0.020064 -1.35425 0.177411    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.702574     Adj. R2: 0.954989
                 Within R2: 8.835e-4

In [7]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(v) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j + i^j,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  i^j: 20,608
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                      -0.063500   0.092210 -0.688639
year2022                                       0.002725   0.108219  0.025184
year2020:log(deaths_i)                        -0.018044   0.008895 -2.028577
year2021:log(deaths_i)                        -0.016126   0.008817 -1.828983
year2022:log(deaths_i)                        -0.016141   0.010470 -1.541676
year2020:log(deaths_j)                        -0.018412   0.009123 -2.018171
year2021:log(deaths_j)                         0.001634   0.009407  0.173663
year2022:log(deaths_j)                         0.005853   0.010745  0.544703
year2020:C8EV_International_travel_controls_i -0.080207   0.030382 -2.639977
year2021:C8EV_International_travel_controls_i -0.031253   0.019420 -1.609341
year2022:C8EV_Int

### Инструментальные переменные на панельных данных

In [15]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 

fe_iv = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j+i^j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175,  i^j: 20,605
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.041512   0.033647 -1.233737 0.218965
fit_log(deaths_j)                     0.009252   0.031410  0.294565 0.768678
C8EV_International_travel_controls_i -0.050164   0.025373 -1.977088 0.049611
C8EV_International_travel_controls_j -0.018634   0.019891 -0.936790 0.350165
                                        
fit_log(deaths_i)                       
fit_log(deaths_j)                       
C8EV_International_travel_controls_i *  
C8EV_International_travel_controls_j    
---
Signif. codes:  0 '***' 

In [16]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 

fe_iv_no_school = feols(log(v) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j+ i^j|
 log(deaths_i) + log(deaths_j) ~ 
H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
+
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
+
average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv_no_school %>% summary(cluster = ~ i+j)

NOTE: 17,571 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/15,000).



TSLS estimation - Dep. Var.: log(v)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : H6M_Facial_Coverings_i, H6M_Facial_Coverings_j, C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j, average_surface_temperature_i, average_surface_temperature_j
Second stage: Dep. Var.: log(v)
Observations: 52,736
Fixed-effects: year: 3,  i: 166,  j: 166,  i^j: 19,208
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
fit_log(deaths_i)                    -0.050296   0.034750 -1.447372 0.149690
fit_log(deaths_j)                    -0.003506   0.029923 -0.117158 0.906877
C8EV_International_travel_controls_i -0.052973   0.026625 -1.989565 0.048291
C8EV_International_travel_controls_j -0.023170   0.022520 -1.028838 0.305062
                                        
fit_log(deaths_i)                       
fit_log(deaths_j)                       


### Экспорт результатов в латех

In [17]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf)

,,fe_no_iv,heterogenous_fe,fe_iv
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v),log(v),log(v)
2,,,,
3,log(deaths_i),-0.0155* (0.0078),,-0.0415 (0.0336)
4,log(deaths_j),-0.0076 (0.0070),,0.0093 (0.0314)
5,C8EV_International_travel_controls_i,-0.0382. (0.0200),,-0.0502* (0.0254)
6,C8EV_International_travel_controls_j,-0.0272 (0.0201),,-0.0186 (0.0199)
7,year2021,,-0.0635 (0.0922),
8,year2022,,0.0027 (0.1082),
9,year2020 x log(deaths_i),,-0.0180* (0.0089),


In [18]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=TRUE)

\begingroup
\centering
\begin{tabular}{lccc}
   \tabularnewline \midrule \midrule
   Dependent Variable: & \multicolumn{3}{c}{log(v)}\\
   Model:                                                          & (1)           & (2)            & (3)\\  
   \midrule
   \emph{Variables}\\
   log(deaths\_i)                                                  & -0.0155$^{*}$ &                & -0.0415\\   
                                                                   & (0.0078)      &                & (0.0336)\\   
   log(deaths\_j)                                                  & -0.0076       &                & 0.0093\\   
                                                                   & (0.0070)      &                & (0.0314)\\   
   C8EV\_International\_travel\_controls\_i                        & -0.0382$^{.}$ &                & -0.0502$^{*}$\\   
                                                                   & (0.0200)      &                & (0.0254)\\   
   C8EV\_International